In [4]:
using Plots
using Statistics
using JSON
using Dates
using HTTP

In [5]:
plotlyjs(); # setting the backend 

In [3]:
include("covidplotutils.jl");

In [42]:
data = JSON.parse(String(HTTP.request("GET", "https://pomber.github.io/covid19/timeseries.json").body));

In [43]:
country="Egypt"
deaths, confirmed, dates, recovered = extract_country_data(country,data);
date_density=5
plot(1:length(dates),deaths, label="deaths", title="Death toll of Covid-19 in $country", legend=:topleft, xrotation=45, xticks=(1:date_density:length(dates),string.(dates[1:date_density:length(dates)])))

<!DOCTYPE html>
 
 
 Plots.jl

In [12]:
death_threshold=5
st=findfirst(x->x>death_threshold,deaths)
range_of_interest=st:length(dates)
deaths, confirmed, dates, recovered =deaths[range_of_interest], 
                                     confirmed[range_of_interest], 
                                     dates[range_of_interest], 
                                     recovered[range_of_interest];

In [13]:
plot(1:length(dates),deaths, ylabel="deaths", label="",  xlabel="Date",
    title="Covid-19 Death toll in $country since $death_threshold deaths", 
    legend=:topleft, xrotation=45, 
    xticks=(1:date_density:length(dates),string.(dates[1:date_density:length(dates)])))

<!DOCTYPE html>
 
 
 Plots.jl

Lets see what happens when we use a logarithmic scale for the y-axis

In [14]:
plot(1:length(dates),deaths, ylabel="deaths", label="",  xlabel="Date",
    title="Covid-19 Death toll in $country since $death_threshold deaths", 
    yscale=:log10,
    legend=:topleft, xrotation=45, 
    xticks=(1:date_density:length(dates),string.(dates[1:date_density:length(dates)])))

<!DOCTYPE html>
 
 
 Plots.jl

In [15]:
y=log.(deaths)
x=[ones(length(dates)) [1:length(dates)...]]
v=x\y

2-element Array{Float64,1}:
 2.1002845582338554
 0.11198576656161567

In [16]:
using Printf
@printf("Daily rate %2.2f",exp(v[2]))

Daily rate 1.12

In [18]:
scatter(1:length(dates),deaths, ylabel="deaths", label="",  xlabel="Date",
    title="Covid-19 Death toll in $country since $death_threshold deaths", 
    yscale=:log10,
    legend=:topleft, xrotation=45, 
    xticks=(1:date_density:length(dates),string.(dates[1:date_density:length(dates)])))
plot!([exp(v[1]+v[2]*x) for x=1:length(dates)], label="prediction")

<!DOCTYPE html>
 
 
 Plots.jl

In [19]:
y=log.(deaths)
x=[ones(length(dates)) [1:length(dates)...]]
v=x\y
p=scatter(1:length(dates),deaths, 
    label="deaths", title="Death toll of Covid-19 in $country", 
    legend=:topleft, xrotation=45, 
    xticks=(1:date_density:length(dates),string.(dates[1:date_density:length(dates)])))
plot!(p,[exp(v[1]+v[2]*x) for x=1:length(dates)], label="model based on overall data")
plot!(p,ann=[(29,300,Plots.text(@sprintf("%2.2f xdaily",exp(v[2])),10, :center,
             p.series_list[2].plotattributes[:linecolor]))], ylim=450)
days_in_past=10
start_point=length(deaths)-days_in_past
y=log.(deaths[start_point:end])
x=[ones(length(start_point:length(dates))) [start_point:length(dates)...]]
v=x\y
model_range=length(dates)-days_in_past:length(dates)+2
plot!(p,model_range,[exp(v[1]+v[2]*x) for x=model_range], label="model based on last $days_in_past days")
plot!(p,ann=[(32,180,Plots.text(@sprintf("%2.2fx daily",exp(v[2])),10, :center,
             p.series_list[3].plotattributes[:linecolor]))], ylim=450)

<!DOCTYPE html>
 
 
 Plots.jl

Social Distancing has been observed to slow the rate os that it is not exponetional... we can now try to make a curve fit using the data for April onwards. 

In [28]:
target_deaths=100_000
@printf(
"It will take %d days to reach %d deaths from today.
We have have doubling period of %d days. 
This is based on a exponential growth model of data based on the  last %d days.",
    days_till_target_number(100_000)[2], target_deaths, days_till_target_number(100_000)[1],days_in_past )

It will take 94 days to reach 100000 deaths from today.
We have have doubling period of 11 days. 
This is based on a exponential growth model of data based on the  last 10 days.

In [29]:
segments, v_interval=segmented_models(deaths)
plot_doubling_times(deaths,v_interval,segments,dates,"Deaths")

<!DOCTYPE html>
 
 
 Plots.jl

This can be compared with the figures in the "[our world in data site](https://ourworldindata.org/coronavirus#global-comparison-where-are-confirmed-deaths-increasing-most-rapidly)" 

In [30]:
plot_whisker_fit(deaths, v_interval,segments,dates, "Death toll")

<!DOCTYPE html>
 
 
 Plots.jl

# Confirmed cases

In [31]:
deaths, confirmed, dates, recovered = extract_country_data(country,data);
date_density=5
plot(1:length(dates),confirmed, label="confirmed", title="Confirmed cases of Covid-19 in $country", legend=:topleft, xrotation=45, xticks=(1:date_density:length(dates),string.(dates[1:date_density:length(dates)])))

<!DOCTYPE html>
 
 
 Plots.jl

We look at at the situation after 100 confirmed cases

In [32]:
st=findfirst(x->x>100, confirmed)
dates=dates[st:end]
confirmed=confirmed[st:end]
scatter(1:length(dates),confirmed, 
    yscale=:log10,
    label="confirmed", title="Confirmed cases of Covid-19 in $country", 
    legend=:topleft, xrotation=45, 
    xticks=(1:date_density:length(dates),string.(dates[1:date_density:length(dates)])))


<!DOCTYPE html>
 
 
 Plots.jl

In [33]:
y=log.(confirmed)
x=[ones(length(dates)) [1:length(dates)...]]
v=x\y
p=scatter(1:length(dates),confirmed, 
    label="confirmed", title="Confirmed cases of Covid-19 in $country", 
    legend=:topleft, xrotation=45, 
    xticks=(1:date_density:length(dates),string.(dates[1:date_density:length(dates)])))
plot!(p, [exp(v[1]+v[2]*x) for x=1:length(dates)], label="model based on overall data")
plot!(p,ann=[(29,3000,Plots.text(@sprintf("%2.2f xdaily",exp(v[2])),10, :center,
             p.series_list[2].plotattributes[:linecolor]))], ylim=450)
days_in_past=10
start_point=length(confirmed)-days_in_past
y=log.(confirmed[start_point:end])
x=[ones(length(start_point:length(dates))) [start_point:length(dates)...]]
v=x\y
model_range=length(dates)-days_in_past:length(dates)+2
plot!(p,model_range,[exp(v[1]+v[2]*x) for x=model_range], label="model based on last $days_in_past days")
plot!(p,ann=[(33,2000,Plots.text(@sprintf("%2.2fx daily",exp(v[2])),10, :center,
             p.series_list[3].plotattributes[:linecolor]))], ylim=450)
days_in_past=5
start_point=length(confirmed)-days_in_past
y=log.(confirmed[start_point:end])
x=[ones(length(start_point:length(dates))) [start_point:length(dates)...]]
v=x\y
model_range=length(dates)-days_in_past:length(dates)+2
plot!(p,model_range,[exp(v[1]+v[2]*x) for x=model_range], label="model based on last $days_in_past days")
plot!(p,ann=[(38,2800,Plots.text(@sprintf("%2.2fx daily",exp(v[2])),10, :center,
             p.series_list[4].plotattributes[:linecolor]))], ylim=450)

<!DOCTYPE html>
 
 
 Plots.jl

We notice here thatrecently the number of confirmed cases have been declining. 
We can look at the doubling times over 5 day intervals. 

In [34]:
segments, v_interval=segmented_models(confirmed);

In [35]:
p=plot_whisker_fit(confirmed, v_interval,segments,dates)

<!DOCTYPE html>
 
 
 Plots.jl

In [37]:
target_confirmed=100_000
tt=days_till_target_number(target_confirmed,confirmed,v_interval[end])
@printf(
"It will take %d days to reach %d confrimed cases from today.
We have have doubling period of %d days. 
This is based on a exponential growth model of data based on the  last %d days.",
    tt[2], target_confirmed, tt[1],5)

It will take 54 days to reach 100000 confrimed cases from today.
We have have doubling period of 11 days. 
This is based on a exponential growth model of data based on the  last 5 days.

In [38]:
plot_doubling_times(confirmed,v_interval,segments,dates,"Confirmed Cases")

<!DOCTYPE html>
 
 
 Plots.jl